In [3]:
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.functional as F

import numpy as np
import pywt
import csv
import os
import pandas as pd
import math
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# data preprocessing
from sklearn.preprocessing import StandardScaler
# data splitting
from sklearn.model_selection import train_test_split

# data modeling
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
#visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [4]:
lead_samples = pd.read_csv("/home/ngsci/datasets/silent-cchs-ecg/csv/lead-samples-count.csv")
rwma_outcomes = pd.read_csv("/home/ngsci/datasets/silent-cchs-ecg/csv/rwma-outcomes.csv")

In [5]:
#merged_df = pd.merge(lead_samples.drop(columns=['holdout']), rwma_outcomes, on='ecg_id')
#merged_df
#merged_dfs_arr = merged_df.values
#print(merged_dfs_arr.shape)


In [16]:
#DATA loading
npy_filepath = "/home/ngsci/datasets/silent-cchs-ecg/npy"
dir_list = os.listdir(npy_filepath)
npy_arrays = []
for each in dir_list:
    file = f"{npy_filepath}/{each}"
    tempf = np.load(file)[:,:,0:5000]  
    npy_arrays.append(tempf)
    
stacked = np.stack(npy_arrays, axis= 0)
stacked = torch.from_numpy(stacked)
stacked = stacked.permute(1,0,2,3)
stacked = stacked.reshape(3750, 12*5000) # log Reg akz. 2D

rwma = pd.read_csv("/home/ngsci/datasets/silent-cchs-ecg/csv/rwma-outcomes.csv")
rwma = rwma.astype(float)
rwma = torch.tensor(rwma.iloc[:,1])

In [17]:
class ekgData():
    def __init__(self):
        self.x = stacked
        self.y = rwma
        self.n_persons = len(self.y)

        # Split the dataset into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(self.x, self.y, test_size=0.2, random_state=42)

        # Further split the training set into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

        self.x_train = X_train
        self.y_train = y_train
        self.x_val = X_val
        self.y_val = y_val
        self.x_test = X_test
        self.y_test = y_test

    def get_train_data(self): #get training data
        # return as tuple
        return self.x_train, self.y_train
        
    def get_val_data(self): #get validation data
        # return as tuple
        return self.x_val, self.y_val
        
    def get_test_data(self): #get testing data
        # return as tuple
        return self.x_test, self.y_test
        
    def __len__(self):
        return self.n_persons


In [18]:
data = ekgData()
x_train, y_train = data.get_train_data()
x_val, y_val = data.get_val_data()
x_test, y_test = data.get_test_data()



In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

lr = LogisticRegression()
model = lr.fit(x_train, y_train)
lr_predict = lr.predict(x_test)
lr_conf_matrix = confusion_matrix(y_test, lr_predict)
lr_acc_score = accuracy_score(y_test, lr_predict)
print("confussion matrix")
print(lr_conf_matrix)
print("\n")
print("Accuracy of Logistic Regression:",lr_acc_score*100,'\n')
print(classification_report(y_test,lr_predict))

confussion matrix
[[356 318]
 [ 32  44]]


Accuracy of Logistic Regression: 53.333333333333336 

              precision    recall  f1-score   support

         0.0       0.92      0.53      0.67       674
         1.0       0.12      0.58      0.20        76

    accuracy                           0.53       750
   macro avg       0.52      0.55      0.44       750
weighted avg       0.84      0.53      0.62       750



/opt/venv/default/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
